# DSE Multi-Company LSTM Trainer (1, 3, 7-day forecasts)

This notebook trains **separate LSTM models per company/scrip** from a single CSV of OHLCV data, 
and for **three forecast horizons**: `n_ahead = 1, 3, 7`.

- Input CSV layout (example):
  ```
  Date,Scrip,Open,High,Low,Close,Volume
  2000-01-01,ACI,12.67,13.77,12.67,13.63,19850
  2000-01-01,ALLTEX,4.7,4.75,4.7,4.7,17500
  ```
- Output: per `Scrip` directory under `models/`:
  - `lstm_{SCRIP}_seq{SEQ_LEN}_nahead{N}.keras` (Keras v3 single-file model)
  - `scaler_{SCRIP}.bin` (MinMaxScaler fit on training data of that Scrip)
  - optional predictions/metrics files

**Notes & choices**
- Uses **MinMaxScaler** fit on the **training split only** to avoid data leakage; applied to test/validation and future inputs via `.transform()`.
- Univariate LSTM on `Close` by default (simplest & common baseline). You can switch to multivariate by changing `FEATURE_COLS`.
- Three distinct models per scrip (for 1, 3, and 7-day horizons). One scaler per scrip.
- Robust to short histories (skips scrips that don't have enough rows).

## 0) (Optional) Install dependencies

Run this **only if you get import errors**. Pin versions as needed for your environment.

In [1]:
# If needed, uncomment and run:
# %pip install -q --upgrade pip
# %pip install -q numpy pandas scikit-learn tensorflow matplotlib joblib

## 1) Imports & Reproducibility

In [2]:
import os
import json
import math
import joblib
import numpy as np
import pandas as pd

from datetime import timedelta

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Reproducibility
def set_global_seed(seed: int = 42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_global_seed(42)

print(tf.__version__)

2025-08-18 12:36:23.267323: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-18 12:36:23.270159: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-18 12:36:23.277287: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755498983.289099   26095 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755498983.292387   26095 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755498983.302324   26095 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

2.19.0


## 2) Configuration

- `CSV_PATH`: path to your **single** CSV with all companies.
- `SCRIP_COLUMN`, `DATE_COLUMN`: column names in your CSV.
- `TARGET_COLUMN`: which column to forecast (default `Close`).
- `SEQ_LEN`: lookback window length (e.g., 60).
- `HORIZONS`: list of forecast horizons to train for (here: `[1, 3, 7]`).

Models and scalers are saved under `SAVE_DIR/models/{Scrip}/`.
Optionally, predictions/metrics go to `SAVE_DIR/predictions/` and `SAVE_DIR/metrics/`.

In [3]:
# === Required: set your CSV path here ===
CSV_PATH = "./dataset/merged_stock_data.csv"  # <- CHANGE THIS to your dataset path

# === Columns ===
DATE_COLUMN = "Date"
SCRIP_COLUMN = "Scrip"
TARGET_COLUMN = "Close"   # forecasting Close

# Optional multivariate: include more features here.
# For baseline univariate LSTM on Close only, leave as ["Close"].
FEATURE_COLS = ["Close"]

# === Sequence & horizons ===
SEQ_LEN = 60
HORIZONS = [1, 3, 7]

# === Splits ===
TRAIN_RATIO = 0.8  # remaining used for testing; validation handled via validation_split in .fit

# === Training hyperparams ===
MAX_EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
PATIENCE = 10  # early stopping
VAL_SPLIT = 0.2

# === Save locations ===
SAVE_DIR = "./artifacts"  # root
MODELS_DIR = os.path.join(SAVE_DIR, "models")
PREDS_DIR = os.path.join(SAVE_DIR, "predictions")
METRICS_DIR = os.path.join(SAVE_DIR, "metrics")

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(PREDS_DIR, exist_ok=True)
os.makedirs(METRICS_DIR, exist_ok=True)

print("Saving to:", os.path.abspath(SAVE_DIR))

Saving to: /mnt/Work/projects/stock_cast/predictor/artifacts


## 3) Data Loading & Utilities
- Reads CSV, parses dates, sorts by `Date`.
- Lists all unique `Scrip` values.
- For each scrip:
  - Splits by date into **train/test** (no shuffling).
  - Fits `MinMaxScaler` on **train** only (to avoid leakage).
  - Builds sliding windows of shape `(samples, SEQ_LEN, n_features)` with targets of shape `(samples, N_AHEAD)`.

In [4]:
from sklearn.preprocessing import MinMaxScaler
from typing import Tuple, Dict, Any

def read_all_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    assert DATE_COLUMN in df.columns, f"Missing column: {DATE_COLUMN}"
    assert SCRIP_COLUMN in df.columns, f"Missing column: {SCRIP_COLUMN}"
    assert TARGET_COLUMN in df.columns, f"Missing column: {TARGET_COLUMN}"
    # Ensure Date type and sorting
    df[DATE_COLUMN] = pd.to_datetime(df[DATE_COLUMN])
    df = df.sort_values([SCRIP_COLUMN, DATE_COLUMN]).reset_index(drop=True)
    return df

def get_scrips(df: pd.DataFrame) -> np.ndarray:
    return df[SCRIP_COLUMN].dropna().unique()

def split_train_test(values_len: int, train_ratio: float) -> Tuple[int, int]:
    train_end = int(values_len * train_ratio)
    test_start = train_end
    return train_end, test_start

def fit_scaler_on_train(train_df: pd.DataFrame) -> MinMaxScaler:
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(train_df[FEATURE_COLS].values)
    return scaler

def scale_df(df: pd.DataFrame, scaler: MinMaxScaler) -> np.ndarray:
    return scaler.transform(df[FEATURE_COLS].values)

def build_sequences(values: np.ndarray, seq_len: int, n_ahead: int) -> Tuple[np.ndarray, np.ndarray]:
    """
    values: scaled array shape (T, n_features). We'll predict TARGET_COLUMN but here we assume FEATURE_COLS includes it.
    For univariate, n_features=1 and both X and y come from the same array.
    """
    X, y = [], []
    T = len(values)
    for i in range(T - seq_len - n_ahead + 1):
        window = values[i:i+seq_len]
        target = values[i+seq_len:i+seq_len+n_ahead, 0]  # use first column as target (Close)
        X.append(window)
        y.append(target)
    return np.array(X), np.array(y)

def prepare_scrip_data(scrip_df: pd.DataFrame, seq_len: int, n_ahead: int, train_ratio: float) -> Dict[str, Any]:
    # split into train/test by date order
    n = len(scrip_df)
    train_end, test_start = split_train_test(n, train_ratio=train_ratio)
    train_df = scrip_df.iloc[:train_end].copy()
    test_df  = scrip_df.iloc[test_start:].copy()

    # fit scaler on TRAIN only
    scaler = fit_scaler_on_train(train_df)

    # transform
    train_scaled = scale_df(train_df, scaler)
    test_scaled  = scale_df(test_df, scaler)

    # build sequences separately to avoid leakage across the split
    X_train, y_train = build_sequences(train_scaled, seq_len, n_ahead)
    X_test, y_test   = build_sequences(test_scaled, seq_len, n_ahead)

    return {
        "scaler": scaler,
        "train_df": train_df,
        "test_df": test_df,
        "X_train": X_train, "y_train": y_train,
        "X_test": X_test, "y_test": y_test,
    }

## 4) Model Builder

Simple, sturdy baseline:
- 2 stacked LSTM layers with dropout
- Dense output of size `N_AHEAD`
- Adam optimizer
- EarlyStopping + ReduceLROnPlateau

In [5]:
def build_lstm_model(input_shape, n_ahead: int, lr: float = LEARNING_RATE) -> keras.Model:
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.LSTM(64, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(32),
        layers.Dropout(0.2),
        layers.Dense(n_ahead)  # outputs N_AHEAD values
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                  loss="mse",
                  metrics=["mae"])
    return model

def get_callbacks():
    es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=PATIENCE, restore_best_weights=True)
    rlrop = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=max(3, PATIENCE//2), min_lr=1e-5)
    return [es, rlrop]

## 5) Training & Saving Helpers

- Trains a model for a single `(Scrip, N_AHEAD)` pair.
- Saves the Keras model and the scaler (only once per Scrip).
- Evaluates on test set (RMSE, MAPE) and saves metrics & optional predictions.

In [6]:
from math import sqrt

def rmse(y_true, y_pred):
    return sqrt(np.mean((y_true - y_pred) ** 2))

def mape(y_true, y_pred, eps=1e-8):
    return np.mean(np.abs((y_true - y_pred) / np.maximum(np.abs(y_true), eps))) * 100.0

def inverse_transform_1d(arr_1d, scaler: MinMaxScaler) -> np.ndarray:
    """Inverse transform a 1D array that represents the TARGET_COLUMN only."""
    return scaler.inverse_transform(np.array(arr_1d).reshape(-1, 1)).ravel()

def train_one_horizon(scrip: str, scrip_df: pd.DataFrame, n_ahead: int, save_root: str) -> Dict[str, Any]:
    # prepare data
    prepared = prepare_scrip_data(scrip_df, SEQ_LEN, n_ahead, TRAIN_RATIO)
    scaler = prepared["scaler"]
    X_train, y_train = prepared["X_train"], prepared["y_train"]
    X_test, y_test   = prepared["X_test"],  prepared["y_test"]

    # Check data sufficiency
    if len(X_train) == 0 or len(X_test) == 0:
        return {
            "scrip": scrip, "n_ahead": n_ahead, "status": "skipped",
            "reason": f"Not enough data after windowing (SEQ_LEN={SEQ_LEN}, N_AHEAD={n_ahead})."
        }

    input_shape = (X_train.shape[1], X_train.shape[2])  # (SEQ_LEN, n_features)

    # build & train
    model = build_lstm_model(input_shape, n_ahead)
    hist = model.fit(
        X_train, y_train,
        epochs=MAX_EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=VAL_SPLIT,
        verbose=0,
        callbacks=get_callbacks(),
        shuffle=False,  # keep order
    )

    # Evaluate on test
    y_pred_scaled = model.predict(X_test, verbose=0)
    # compute metrics per step across the horizon
    # inverse transform each horizon step separately
    # both y_test and y_pred_scaled have shape (samples, n_ahead)
    metrics = {}
    rmse_list, mape_list = [], []
    for step in range(n_ahead):
        y_true_step = inverse_transform_1d(y_test[:, step], scaler)
        y_pred_step = inverse_transform_1d(y_pred_scaled[:, step], scaler)
        rmse_list.append(rmse(y_true_step, y_pred_step))
        mape_list.append(mape(y_true_step, y_pred_step))
    metrics["rmse_per_step"] = rmse_list
    metrics["mape_per_step"] = mape_list
    metrics["rmse_mean"] = float(np.mean(rmse_list))
    metrics["mape_mean"] = float(np.mean(mape_list))

    # Save model & scaler (scaler once per scrip)
    scrip_dir = os.path.join(save_root, scrip)
    os.makedirs(scrip_dir, exist_ok=True)

    model_path = os.path.join(scrip_dir, f"lstm_{scrip}_seq{SEQ_LEN}_nahead{n_ahead}.keras")
    model.save(model_path)

    scaler_path = os.path.join(scrip_dir, f"scaler_{scrip}.bin")
    if not os.path.exists(scaler_path):
        joblib.dump(scaler, scaler_path)

    # Optional: save latest forecast (next n_ahead days) using last window of ALL data for that scrip
    # (fit scaler on train only already; we transform full scrip df with the same scaler)
    full_scaled = scaler.transform(scrip_df[FEATURE_COLS].values)
    if len(full_scaled) >= SEQ_LEN:
        last_window = full_scaled[-SEQ_LEN:]
        last_window = last_window.reshape(1, SEQ_LEN, full_scaled.shape[1])
        next_scaled = model.predict(last_window, verbose=0).ravel()
        next_unscaled = inverse_transform_1d(next_scaled, scaler)
        forecast_df = pd.DataFrame({
            "step_ahead": np.arange(1, n_ahead+1),
            "prediction": next_unscaled
        })
        forecast_path = os.path.join(scrip_dir, f"forecast_next_{n_ahead}.csv")
        forecast_df.to_csv(forecast_path, index=False)

    # Return record
    return {
        "scrip": scrip,
        "n_ahead": n_ahead,
        "status": "ok",
        "model_path": model_path,
        "scaler_path": scaler_path,
        "metrics": metrics,
    }

## 6) Orchestration: Train all companies for 1/3/7-day horizons

In [7]:
# Load full dataset
df_all = read_all_data(CSV_PATH)

# Keep only needed columns
needed_cols = [DATE_COLUMN, SCRIP_COLUMN] + list(set(FEATURE_COLS + [TARGET_COLUMN]))
df_all = df_all[needed_cols].copy()

# For univariate Close-only, ensure FEATURE_COLS[0] == TARGET_COLUMN
if FEATURE_COLS[0] != TARGET_COLUMN:
    print("Note: Using multivariate features. TARGET_COLUMN must be among FEATURE_COLS; "
          "the first feature is treated as the target for inverse scaling.")

scrips = get_scrips(df_all)
print(f"Found {len(scrips)} scrips.")

records = []
for idx, scrip in enumerate(scrips, 1):
    scrip_df = df_all[df_all[SCRIP_COLUMN] == scrip].sort_values(DATE_COLUMN).reset_index(drop=True)

    # Basic NA handling
    scrip_df = scrip_df.dropna(subset=FEATURE_COLS)  # drop rows with missing features
    if len(scrip_df) < (SEQ_LEN + max(HORIZONS) + 10):  # conservative minimum
        print(f"[{idx}/{len(scrips)}] Skipping {scrip}: not enough rows ({len(scrip_df)}).")
        continue

    print(f"[{idx}/{len(scrips)}] Training {scrip} ...")
    for n_ahead in HORIZONS:
        result = train_one_horizon(scrip, scrip_df, n_ahead, save_root=MODELS_DIR)
        records.append({
            "scrip": scrip,
            "n_ahead": n_ahead,
            "status": result.get("status"),
            "model_path": result.get("model_path"),
            "scaler_path": result.get("scaler_path"),
            "rmse_mean": None if "metrics" not in result else result["metrics"]["rmse_mean"],
            "mape_mean": None if "metrics" not in result else result["metrics"]["mape_mean"],
        })
        if result.get("status") != "ok":
            print(f"  - {scrip} n_ahead={n_ahead}: {result.get('reason')}")
        else:
            print(f"  - {scrip} n_ahead={n_ahead}: OK")

# Save a CSV summary of metrics/paths
summary_df = pd.DataFrame(records)
summary_path = os.path.join(METRICS_DIR, "training_summary.csv")
summary_df.to_csv(summary_path, index=False)
print("Saved summary to:", summary_path)

summary_df.tail(10)

Found 696 scrips.
[1/696] Training 00DS30 ...


2025-08-18 12:36:26.070769: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


  - 00DS30 n_ahead=1: OK
  - 00DS30 n_ahead=3: OK
  - 00DS30 n_ahead=7: OK
[2/696] Training 00DSES ...
  - 00DSES n_ahead=1: OK
  - 00DSES n_ahead=3: OK
  - 00DSES n_ahead=7: OK
[3/696] Training 00DSEX ...
  - 00DSEX n_ahead=1: OK
  - 00DSEX n_ahead=3: OK
  - 00DSEX n_ahead=7: OK
[4/696] Training 00DSMEX ...
  - 00DSMEX n_ahead=1: OK
  - 00DSMEX n_ahead=3: OK
  - 00DSMEX n_ahead=7: OK
[5/696] Training 1JANATAMF ...
  - 1JANATAMF n_ahead=1: OK
  - 1JANATAMF n_ahead=3: OK
  - 1JANATAMF n_ahead=7: OK
[6/696] Training 1STPRIMFMF ...
  - 1STPRIMFMF n_ahead=1: OK
  - 1STPRIMFMF n_ahead=3: OK
  - 1STPRIMFMF n_ahead=7: OK
[7/696] Training AAMRANET ...
  - AAMRANET n_ahead=1: OK
  - AAMRANET n_ahead=3: OK


KeyboardInterrupt: 

## 7) How to Load a Trained Model Later (Example)

This shows how your **website/backend** can load a specific Scrip + horizon, 
scale the last 60 days, and predict the next *n* days.

In [ ]:
# Example: replace with an actual scrip from your data
EXAMPLE_SCRIP = "ACI"  # e.g., "ACI"

if EXAMPLE_SCRIP is not None:
    scrip_dir = os.path.join(MODELS_DIR, EXAMPLE_SCRIP)
    # Choose horizon
    n_ahead = 7
    model_path = os.path.join(scrip_dir, f"lstm_{EXAMPLE_SCRIP}_seq{SEQ_LEN}_nahead{n_ahead}.keras")
    scaler_path = os.path.join(scrip_dir, f"scaler_{EXAMPLE_SCRIP}.bin")

    model = keras.models.load_model(model_path)
    scaler = joblib.load(scaler_path)

    # Prepare the last window from the CSV (same flow as training)
    scrip_df = df_all[df_all[SCRIP_COLUMN] == EXAMPLE_SCRIP].sort_values(DATE_COLUMN).reset_index(drop=True)
    scaled = scaler.transform(scrip_df[FEATURE_COLS].values)
    last_window = scaled[-SEQ_LEN:].reshape(1, SEQ_LEN, scaled.shape[1])
    pred_scaled = model.predict(last_window, verbose=0).ravel()
    pred = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).ravel()

    print(f"Next {n_ahead} predictions for {EXAMPLE_SCRIP}:", pred)
else:
    print("Set EXAMPLE_SCRIP to test loading & predicting.")

---

### Tips
- If you want **multivariate** input (e.g., `['Open','High','Low','Close','Volume']`), set `FEATURE_COLS` accordingly.
  - The **first** entry in `FEATURE_COLS` is treated as the target for inverse scaling (keep `Close` first).
- Consider **walk-forward validation** for more robust evaluation.
- For deployment at scale, consider exporting to **SavedModel** and serving via TensorFlow Serving.

### Done!